This project takes in a CSV of transactions from [Intuit's Mint, Expenses Tracking App](https://mint.intuit.com) and summarizes cash flow into a Sankey visual, i.e.:
* exports a txt that can be used to generate a Sankey diagram at [SankeyMatic](https://sankeymatic.com/build)
* displays a sankey plot using python plotly (not as beautiful imo)

In [ ]:
# Import dependenciess
import yaml
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# Load Dataset
transactions_csv = ""
df = pd.read_csv(transactions_csv)


In [ ]:
# Load a map of Category -> (Main Category, Sub Category) so we can have more granular categories than what is in Mint's CSV export
# Shopping -> (Shopping, None)
# Clothing -> (Shopping, Clothing)

main_sub_categories = {}

with open("mint-categories.yaml", "r") as rstream:
    for main, subs in yaml.safe_load(rstream).items():
        main_sub_categories[main] = (main, None)
        if subs:
            for sub_category in subs:
                main_sub_categories[sub_category] = (main, sub_category)

main_sub_categories


In [ ]:
# Using the map, add Main Category and Sub Category fields to every row (transaction)

df["Main Category"] = df.apply(
    lambda row: main_sub_categories[row["Category"]][0], axis=1
)

df["Sub Category"] = df.apply(
    lambda row: main_sub_categories[row["Category"]][1], axis=1
)


In [ ]:
# Filter out unwanted categories and accounts

unwanted_accounts = []
unwanted_categories = [
    "Transfer",
]

is_unwanted_accounts = df["Account Name"].isin(unwanted_accounts)
is_unwanted_categories = df["Main Category"].isin(unwanted_categories)

df = df[~is_unwanted_accounts & ~is_unwanted_categories]


In [ ]:
# Calculate the sums for each Sub Category

# Note: Make debits negative so sums are accurate
df.loc[df["Transaction Type"] == "debit", "Amount"] *= -1

# Note: Some transactions can have only Main Category (Sub Category = NA). To avoid summing these all up together (they belong to different categories and should be summed separately), we also group by Main Category
sums = df.groupby(["Main Category", "Sub Category"], dropna=False, as_index=False)[
    "Amount"
].sum(numeric_only=True)

sums


In [ ]:
# Create a data-structure to store the sums of each Main Category and Sub Category
# {
#   'inflow': {
#       [main_category]: {
#           [sub_category]: {
#               sum: 10
#           },
#       },
#   },
#   'outflow': {
#       [main_category]: {
#           [sub_category]: {
#               sum: 10
#           }
#       },
#   }
# }
cash_flow = {"inflow": {}, "outflow": {}}

# Note: Inflow categories tend to be fewer, so use that list to separate inflow (income) from outflow (expenses) transactions
inflow_categories = ["Income", "Taxes"]

# Fill the data-structure with the sums of each subcategory
for i, row in sums.iterrows():
    amount = row["Amount"]
    mc = row["Main Category"]
    sc = row["Sub Category"]

    if pd.isnull(sc):
        # For transactions with only Main Category and no Sub Category, label them as "<Main Category> (uncategorized)"
        sc = f"{mc} (uncategorized)"

    flow_type = "inflow" if mc in inflow_categories else "outflow"

    if mc not in cash_flow[flow_type]:
        cash_flow[flow_type][mc] = {sc: {"sum": abs(round(amount, 2))}}
    else:
        cash_flow[flow_type][mc][sc] = {"sum": abs(round(amount, 2))}


# Calculate the sums for each Main Category (i.e sum of all Sub Categories under each)
for flow in cash_flow:
    mc_total = 0
    for mc in cash_flow[flow]:
        mc_total = sum([sc["sum"] for _, sc in cash_flow[flow][mc].items()])
        cash_flow[flow][mc]["sum"] = round(mc_total, 2)


cash_flow


In [ ]:
# Transform the cash flow data-structure to sankey visual


def cashflow_to_sankeymatic(cash_flow):
    # Sankeymatic (https://sankeymatic.com/build) uses a text-based format like:
    # Source [amount(absolute)] Sink
    output = ""

    # Route all inflows as sources: <inflow> -> Net Income
    for mc in cash_flow["inflow"]:
        if mc != "sum":
            amount = cash_flow["inflow"][mc]["sum"]
            output += f"{mc} [{amount}] Net Income\n"

    # Route all outflows as sinks: Net Income -> <outflow>
    for mc in cash_flow["outflow"]:
        if mc != "sum":
            mc_total = cash_flow["outflow"][mc]["sum"]
            output += f"Net Income [{mc_total}] {mc}\n"

        # Route all sub categories as sinks of main categories: <main category> -> <sub category>
        for sc in cash_flow["outflow"][mc]:
            if sc != "sum":
                amount = cash_flow["outflow"][mc][sc]["sum"]
                output += f"{mc} [{amount}] {sc}\n"
    return output


def cashflow_to_plotly(cash_flow):
    sources = []
    targets = []
    amounts = []

    labels_index = {"Net Income": 0}
    labels = ["Net Income"]

    def get_label_index(label):
        if label not in labels_index:
            labels.append(label)
            labels_index[label] = len(labels) - 1
        return labels_index[label]

    # Route all inflows as sources: <inflow> -> Net Income
    for mc in cash_flow["inflow"]:
        if mc != "sum":
            sources.append(get_label_index(mc))
            amounts.append(cash_flow["inflow"][mc]["sum"])
            targets.append(0)

    # Route all outflows as sinks: Net Income -> <outflow>
    for mc in cash_flow["outflow"]:
        if mc != "sum":
            sources.append(0)
            amounts.append(cash_flow["outflow"][mc]["sum"])
            targets.append(get_label_index(mc))

        # Route all sub categories as sinks of main categories: <main category> -> <sub category>
        for sc in cash_flow["outflow"][mc]:
            if sc != "sum":
                sources.append(get_label_index(mc))
                amounts.append(cash_flow["outflow"][mc][sc]["sum"])
                targets.append(get_label_index(sc))

    fig = go.Figure(
        data=[
            go.Sankey(
                arrangement="fixed",
                node=dict(
                    pad=15,
                    thickness=5,
                    line=dict(color="black", width=0.5),
                    label=labels,
                    color="blue",
                ),
                link=dict(source=sources, target=targets, value=amounts),
            )
        ]
    )

    fig.update_layout(title_text="Sankey Diagram of Cash Flow", width=1000, height=1200)
    fig.show()


sankey = cashflow_to_sankeymatic(cash_flow)
with open("sankeymatic-input.txt", "w") as rstream:
    rstream.write(sankey)

cashflow_to_plotly(cash_flow)
